<a href="https://colab.research.google.com/github/brunoesperante73/MARINA-model/blob/main/MARINA_(Maritime_Artificial_Routing_Intelligence_%26_Navigation_Analysis).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**MARINA**
##maritime artificial routing intelligence & navigation analysis

In [22]:
# Configuração das APIs
API_KEY_WEATHER = "SUA_CHAVE_OPENWEATHER"
API_KEY_MARINETRAFFIC = "SUA_CHAVE_MARINETRAFFIC"
WEATHER_URL = "https://api.openweathermap.org/data/2.5/weather"
AIS_URL = "https://services.marinetraffic.com/api/exportvessel/v:5/"

# Estilo gráfico inspirado no escritório 300000kms
def plot_map(dataframe, ships, route):
    fig, ax = plt.subplots(figsize=(12, 8))
    dataframe.plot(ax=ax, color='lightgray', edgecolor='white', linewidth=0.5)

    for ship in ships:
        plt.scatter(ship["lon"], ship["lat"], color='red', s=40, label=ship["name"][:10])

    if route is not None:
        plt.plot(route[:, 1], route[:, 0], color='blue', linewidth=2, linestyle='dashed', label="Rota Prevista")

    plt.title("Tráfego Marítimo e Rotas Previstas", fontsize=14, fontweight='bold')
    plt.axis("off")
    plt.show()

# Função para obter dados climáticos reais
def get_weather_data(lat, lon):
    params = {"lat": lat, "lon": lon, "appid": API_KEY_WEATHER, "units": "metric"}
    response = requests.get(WEATHER_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        wind_speed = data["wind"]["speed"]
        wave_height = data.get("waves", {}).get("height", 2.0)
        return wind_speed, wave_height
    return 10.0, 2.0

# Função para obter tráfego marítimo
def get_ais_data(lat, lon, radius=10):
    params = {
        "key": API_KEY_MARINETRAFFIC,
        "lat": lat,
        "lon": lon,
        "radius": radius,
        "format": "json"
    }
    response = requests.get(AIS_URL, params=params)
    if response.status_code == 200:
        return response.json()
    return []

# Simulação de um dataset expandido (latitude, longitude, velocidade, clima, tempo de viagem)
route_data = np.array([
    [12.34, -45.67, 18.0, 2.5, 5.2],
    [14.56, -47.89, 22.0, 1.8, 4.8],
    [16.78, -50.12, 15.0, 3.2, 6.1],
])

# Criando um modelo mais robusto de aprendizado de máquina
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(3,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')

# Treinar modelo com dados simulados
X_train = route_data[:, :3]
y_train = route_data[:, 4]

model.fit(X_train, y_train, epochs=100, verbose=0)

# Função para prever tempo de viagem
def predict_travel_time(lat, lon, speed, weather):
    return model.predict(np.array([[lat, lon, speed, weather]]))[0, 0]

# Função para gerar uma rota dinâmica considerando tráfego marítimo e clima
def generate_dynamic_route(lat, lon, speed, wind_speed, wave_height):
    delta_lat = np.random.uniform(-0.1, 0.1, size=10)
    delta_lon = np.random.uniform(-0.1, 0.1, size=10)
    return np.column_stack((lat + delta_lat, lon + delta_lon))

import os
import geopandas as gpd

# Download the Natural Earth low resolution dataset
!wget https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_countries.zip
!unzip ne_110m_admin_0_countries.zip

# Get the current working directory
current_directory = os.getcwd()

# Construct the full path to the shapefile
shapefile_path = os.path.join(current_directory, "ne_110m_admin_0_countries.shp")

# Load the shapefile using geopandas with the full path
world = gpd.read_file(shapefile_path)

# Entrada de coordenadas e velocidade
lat = 15.0
lon = -48.0
speed = 20.0

# Obter dados climáticos reais
wind_speed, wave_height = get_weather_data(lat, lon)

# Obter tráfego marítimo real
ais_ships = get_ais_data(lat, lon)

# Gerar rota otimizada
dynamic_route = generate_dynamic_route(lat, lon, speed, wind_speed, wave_height)

# Prever tempo de viagem
predicted_time = predict_travel_time(lat, lon, speed, wind_speed)
print(f"Tempo estimado para ({lat}, {lon}) com {speed} nós, vento {wind_speed}m/s, ondas {wave_height}m: {predicted_time:.2f} horas")

# Exibir dados no mapa GIS
plot_map(world, ais_ships, dynamic_route)

# Criar um painel interativo com Folium
m = folium.Map(location=[lat, lon], zoom_start=5)
folium.Marker([lat, lon], popup=f"Ponto Inicial - {predicted_time:.2f}h").add_to(m)
for ship in ais_ships:
    folium.CircleMarker([ship["lat"], ship["lon"]], radius=5, color='red', fill=True, fill_opacity=0.7, popup=ship["name"]).add_to(m)

# Adicionar a rota otimizada ao mapa interativo
for i in range(len(dynamic_route) - 1):
    folium.PolyLine([dynamic_route[i], dynamic_route[i + 1]], color='blue', weight=2.5, opacity=0.7).add_to(m)

# Salvar mapa
m.save("marina_map.html")


--2025-03-12 15:44:38--  https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/110m/cultural/ne_110m_admin_0_countries.zip
Resolving www.naturalearthdata.com (www.naturalearthdata.com)... 50.87.253.14
Connecting to www.naturalearthdata.com (www.naturalearthdata.com)|50.87.253.14|:443... connected.
HTTP request sent, awaiting response... 500 Internal Server Error
2025-03-12 15:44:38 ERROR 500: Internal Server Error.

unzip:  cannot find or open ne_110m_admin_0_countries.zip, ne_110m_admin_0_countries.zip.zip or ne_110m_admin_0_countries.zip.ZIP.


DataSourceError: /content/ne_110m_admin_0_countries.shp: No such file or directory